# Part 2. Modelling the behavioral data

In Part 1, we had a look at options for preprocessing fMRI data. Now, let's turn to the other essential ingredient of model-based fMRI-analyses: modelling behavioral data.

There are Python packages that allows you to easily fit decision-making models such as the drift diffusion model (DDM). Most notably, [`hddm`](http://ski.clps.brown.edu/hddm_docs/) developed at Brown University is a widely used package.

However, `EMC` (as used in the tutorials earlier this week) has some advantages. For example, `EMC` offers much more models (including a multitude of LBA-model variants, racing diffusion models) and it offers a better sampler. So you might want to use `EMC` to fit a model, and then use `Nipype` to create your fMRI-processing pipeline.

So, we're just going to use `EMC` to fit the behavioral data for now. Note that _this_ notebook you're looking at right now (`Part 2. Modelling the behavioral data`) is actually _not_ an interactive Python notebook ('.ipynb'), but it is an interactive **R** notebook ('.irnb')! That means that instead of running Python code, we can run R-code in here.

For example, to draw some random numbers from a normal distribution (4,1), we can use this R-function:

In [ ]:
rnorm(10, 4, 1)

# note further that the help-function works as usual:
# ?rnorm

# this increases the plotting window size - this line only needs to be run once
options(repr.plot.width = 18, repr.plot.height = 10)

# and plotting is done in-line
plot(1:10, 1:10, main='In line plot')

### So let's use EMC to fit the behavioral data

First, set-up some directories, so R knows where to find EMC, the data, and where to store the results of these analyses (the 'res(ults)Dir')


In [ ]:
## run this first once to set-up some thing for this server -- can forget about this
library(RhpcBLASctl)
omp_set_num_threads(1)
blas_set_num_threads(1)

In [ ]:
projDir <- '/home/neuro/model_based_fmri'
dataDir <- file.path('/data', 'bids')
resDir <- file.path('/data', 'behavior_fits')
dir.create(resDir, showWarnings=FALSE)

# Load EMC
library(EMC2)

##### Load the data, and reformat to a EMC-approved form

In [ ]:
# Load data
dat <- read.csv(file.path(dataDir, 'behavior.tsv'), sep='\t')

# Remove unnecessary columns, rename according to EMC's wishes
dat <- dat[,c('pp', 'cond', 'stimulus', 'response', 'RT')]
colnames(dat) <- c('subjects', 'E', 'S', 'R', 'rt')  # S = stimulus, E = emphasis, R = response, rt = response time

# Remove all extremely fast responses (<.15s)
dat <- dat[dat$rt>.15,]

# Ensure the following columns are of type factor
dat$subjects <- factor(dat$subjects)
dat$E <- factor(dat$E, levels=c('spd', 'acc'))
dat$S <- ifelse(dat$S=='stimright', 'right', 'left')
dat$S <- factor(dat$S, levels=c('left', 'right'))
dat$R <- factor(dat$R, levels=c('left', 'right'))

##### The next step is to set up the model specification

In [ ]:
plot_defective_density(dat,factors=c('E', "S"),layout=c(2,2))

In [ ]:
Emat <- matrix(c(-1,1),ncol=1)
dimnames(Emat) <- list(NULL,"acc-spd")     # estimate the contrast between ACC-SPD trials

# Average rate = intercept, and rate d = difference (match-mismatch) contrast
ADmat <- matrix(c(-1/2,1/2),ncol=1,dimnames=list(NULL,"d"))

### Next, we fit a series of models
Unfortunately the behavioral data in this dataset is rather limited in trial numbers (only 80).
As a result, models with more complexity than 1 parameter varying between SAT conditions, do not sample very well, rendering the resulting parameters rather unreliable.
Hence, here we just do a simple model comparison that compares three models that allow B, v, and t0 to vary between conditions respectively.
In the model that only allows B to vary between conditions, we further constrain `B_{spd}` to 0.05 and only estimate the contrast `B_Eacc-spd`; freely estimating `B_{spd}` also led to sampling issues.


In [ ]:
# convenience function to fit a model and return the samples and posterior predictives
fit_model <- function(data, design, fileName) {
    samplers <- make_emc(data,design,type="standard",rt_resolution=.02)
    if(file.exists(fileName)) {
        samplers <- EMC2:::loadRData(fileName)
        return(samplers)
    }
    samplers <- fit(samplers, fileName=fileName, iter=1000, cores_per_chain=8, cores_for_chains=3, verbose=TRUE, verboseProgress=TRUE, max_tries = 20)
}

# Only B affected by E
design_B <- design(
  factors=list(subjects=levels(dat$subjects),S=levels(dat$S),E=levels(dat$E)),
  Rlevels=levels(dat$R),matchfun=function(d)d$S==d$lR,
  contrasts=list(lM=ADmat,lR=ADmat,S=ADmat,E=Emat),
  formula=list(v~lM,sv~1,B~E,A~1,t0~1),       # <-- only B affected by E
  constants=c(sv=log(1), B=log(0.05)),
  model=LBA)
fit_model(dat, design_B, "samples/sJoN_B.RData")

# Only v affected by E
design_v <- design(
  factors=list(subjects=levels(dat$subjects),S=levels(dat$S),E=levels(dat$E)),
  Rlevels=levels(dat$R),matchfun=function(d)d$S==d$lR,
  contrasts=list(lM=ADmat,lR=ADmat,S=ADmat,E=Emat),
  formula=list(v~E+lM,sv~1,B~1,A~1,t0~1),     # <-- only v affected by E
  constants=c(sv=log(1)),
  model=LBA)
fit_model(dat, design_v, "samples/sJoN_v.RData")


# only t0 affected by E
design_t0 <- design(
  factors=list(subjects=levels(dat$subjects),S=levels(dat$S),E=levels(dat$E)),
  Rlevels=levels(dat$R),matchfun=function(d)d$S==d$lR,
  contrasts=list(lM=ADmat,lR=ADmat,S=ADmat,E=Emat),
  formula=list(v~lM,sv~1,B~1,A~1,t0~E),       # <-- only t0 affected by E
  constants=c(sv=log(1)),
  model=LBA)
fit_model(dat, design_t0, "samples/sJoN_t0.RData")

##### Sampling can take a bit of time, so I ran these models in advance, so you can just load the samplers & posterior predictives

In [ ]:
sJoN_B <- EMC2:::loadRData('./samples/sJoN_B.RData')
sJoN_v <- EMC2:::loadRData('./samples/sJoN_v.RData')
sJoN_t0 <- EMC2:::loadRData('./samples/sJoN_t0.RData')

# Model comparison
Let's start by doing a very basic model comparison based on information criteria (BPIC and DIC)

In [ ]:
compare(list(B=sJoN_B,
             v=sJoN_v,
             t0=sJoN_t0))

Of these three models, the threshold-only model clearly wins both according to the DIC and BPIC.

# Did sampling go well?
Let's then assume the threshold-only model is the best model. What do the chains of the corresponding sampler look like?

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 20)  # here we adjust the plotting area in jupyter; you can adjust the width and height if this doesn't fit your screen well.
plot(sJoN_B, selection='LL', layout=c(4,5))    # this actually plots; i.c. the posterior log likelihoods per subject

The LL plots look like "big hairy caterpillars", as intended, so that's good. Next we can check the posteriors of the parameters at the subject level (alpha)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)  # here we adjust the plotting area in jupyter; you can adjust the width and height if this doesn't fit your screen well.
plot(sJoN_B, selection='alpha', layout=c(1,5), subject='197')

Generally, `t0`, `v_lMd`, and `B_Eacc-spd` seem fine. `v` seems to sample a bit worse, the shapes suggest autocorrelations. What about the population level?

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)  # here we adjust the plotting area in jupyter; you can adjust the width and height if this doesn't fit your screen well.
plot(sJoN_B, selection='mu', layout=c(1,5))

That looks a bit similar for `v` as in the subject level. 

So let's check out how autocorrelated the chains arse. How many effective samples do we have?

In [ ]:
check(emc=sJoN_B)

so we have at least 350 effective samples for v (group-level); which isn't too much but good enough; and Rhat is all close to 1.01, so mixing is sufficient

# Fits
Next, let's visualize how well this model actually fits the data. E.g., are there any obvious misfits? Does the model generally describe the data well? Can it capture the difference between SAT conditions?

In [ ]:
ppJoN_B <- predict(sJoN_B, n_cores=19)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 10)                                    # change plotting area size a bit again
plot_fit(dat,ppJoN_B,layout=c(2,2),factors=c("E","S"),lpos="right",xlim=c(.25,1.5))     # aggregate over subjects and plot 'overall' CDFs

The overall (across-subject) fit seem to look reasonable (but not great). The model seems to capture the median/mean RTs and accuracy okay. Note that individual fits look a lot worse:

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 5)                              #  
plot_fit(dat, ppJoN_B, layout=c(1,2), factors=c("subjects", "E", "S"))           # plot by subject

Given the limited amount of behavioral data, this is to be expected. Most importantly for the purposes of the current tutorial is that the SAT manipulation seems to be captured reasonably well by the threshold contrast `B_Eacc-spd`

# What about the parameters? 
Is `B_Eacc-spd` 'significant' at the population level? Let's first just visualize the posteriors

In [ ]:
plot_pars(sJoN_B,layout=c(1,6),selection="mu") 
# Note that the red lines are the priors, so we can see here that the posteriors really updated (ie we didn't just sample from the prior)

We can also calculate the proportion of density above/below 0 for the `B_Eacc-spd` parameter

In [ ]:
credible(sJoN_B, "B_Eacc-spd", selection='mu', mu=0)

And get the Savage-Dickey ratio at 0

In [ ]:
hypothesis(sJoN_B,"B_Eacc-spd", selection='mu')

So pretty much the entire posterior lies above 0, indicating that there is an effect, and the Bayes Factor for that effect is extremely high.

# In sum
we have a model that: 
- wins simple model comparison, 
- shows reasonably good sampling properties, 
- fits the behavioral data reasonably well, and 
- shows a between-condition contrast in the threshold parameter, captured by `B_Eacc-spd`

For the model-based fMRI analyses, we need the individual subject threshold-difference values (`B_Eacc-spd`), so let's extract these and save

In [ ]:
parameters_df = parameters(sJoN_B, mapped=TRUE, selection="alpha")

median_parameters = aggregate(.~subjects, parameters_df, median)   # this calculates all parameters' median by subject
median_parameters

# and save
write.csv(median_parameters, file=file.path(resDir, 'parameters_per_subject_lba_B_emc2_group2.csv'))